In [1]:
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 14.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 15.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.0/417.0 kB 3.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 12.3 MB/s eta 0:00:0000:0100:01


In [3]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 904.1 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. Th

In [4]:
                                                                                     
import re
import pandas as pd
from tqdm import tqdm
import time

from langchain_community.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain_community.document_loaders.csv_loader import CSVLoader
#from dotenv import load_dotenv
from langchain.chains import LLMChain
from langchain_community.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM
)

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, MistralForCausalLM
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model #prepare_model_for_int8_training
from datasets import load_dataset
import transformers

from huggingface_hub import hf_hub_download
import torch

import argparse
import sys, csv
csv.field_size_limit(sys.maxsize)

2024-05-29 12:35:00.772155: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-29 12:35:00.772262: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-29 12:35:00.930060: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


131072

# Create database

In [10]:
!pip install sentence-transformers faiss-gpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 17.3 MB/s eta 0:00:00:00:0100:01


In [19]:
kbpath = '/kaggle/input/refomed/test_kb_petite.csv'
ENC = 'Lajavaness/sentence-camembert-large'
enc = 'camem'
question = '/kaggle/input/refomed/refomed_test.csv'

In [12]:
loader = CSVLoader(
        file_path=kbpath, source_column="title",
        csv_args={
            "delimiter": "\t",
            #"quotechar": '"',
            "fieldnames": ["index", "title", "text", "length"],
            },
        )
page_content_column = "text"
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

# 'data' holds the text you want to split, splitthe text into documents using the text splitter.
docs = text_splitter.split_documents(data)

modelPath = ENC
#modelPath = "sentence-transformers/all-MiniLM-L6-v2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cuda'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
    )

db = FAISS.from_documents(docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 3})
print("database ready!")

database ready!


In [13]:
def remove_urls(text):
    # Regex pattern to match http and https URLs
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    # Substitute matched URLs with an empty string
    cleaned_text = re.sub(url_pattern, '', text)
    return cleaned_text


def prepare_context(docs):
    cleaned_context, refs = [],[]
    for d in docs:
        raw_d = d.page_content.split('text:')[-1]
        refs.append(str(d.metadata['row']))
        cleaned_d = remove_urls(raw_d)
        #print(raw_d ,"|", cleaned_d.split('length:')[0])
        cleaned_context.append(cleaned_d.split('length:')[0])

    return ' '.join(cleaned_context).strip().replace('\n', ''), '-'.join(refs)

prepare_context(docs)
if 'test' in question:
    split='test'
else:
    split='val'

test_df = pd.read_csv(question, sep='\t')

In [24]:
base_prompt = "Expliquez-moi le terme médical en mots simples, par une paraphrase ou une courte définition :"

complete_prompts = []
src_docs = []
for term in tqdm(test_df.term):
    retrieved_docs = retriever.get_relevant_documents(base_prompt+f' {term}')
    cleaned_context, r = prepare_context(retrieved_docs)

    complete_prmpt = f"""[INST] Vous êtes un expert en médecine. Utilisez les informations suivantes : [{cleaned_context}] {base_prompt}{term} [/INST]"""
    complete_prompts.append(complete_prmpt)
    src_docs.append(r)

print("Complete prompt ready!")


100%|██████████| 1253/1253 [01:15<00:00, 16.49it/s]

Complete prompt ready!


In [16]:
model_name = "LoneStriker/BioMistral-7B-SLERP-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto", # automatically figures out how to best use CPU + GPU for loading model
                                             trust_remote_code=False, # prevents running custom model files on your machine
                                             revision="main")


tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)


config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.57G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:4225: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [29]:
from datasets import Dataset

testdata = Dataset.from_pandas(pd.DataFrame(complete_prompts, columns=['prompt']))

tokenizer.pad_token = tokenizer.eos_token
tokenizer.truncation_side = "right"
tokenizer.padding_side = "left" 
def tokenize_function(examples):
    # extract text
    text = examples["prompt"]
    #tokenize and truncate text
    #tokenizer.truncation_side = "right"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        #padding=True,
        padding='max_length',
        max_length=512
    )
    return tokenized_inputs

tokenized_testdata = testdata.map(tokenize_function, batched=True)
print(tokenized_testdata)
#print(tokenized_testdata[4])

# Define a collate function to pad inputs dynamically
def collate_fn(batch):
    input_ids = torch.nn.utils.rnn.pad_sequence([torch.tensor(item['input_ids']) for item in batch], batch_first=True)
    attention_mask = torch.nn.utils.rnn.pad_sequence([torch.tensor(item['attention_mask']) for item in batch], batch_first=True)
    return {'input_ids': input_ids, 'attention_mask': attention_mask}

batch_size = 32  # Define your batch size
from torch.utils.data import DataLoader
data_loader = DataLoader(tokenized_testdata, batch_size=batch_size, collate_fn=collate_fn)
print("dataloader ready....")


Map:   0%|          | 0/1253 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'input_ids', 'attention_mask'],
    num_rows: 1253
})
dataloader ready....


In [30]:
tokenized_testdata[0]

{'prompt': "[INST] Vous êtes un expert en médecine. Utilisez les informations suivantes : [Anatomopathologie  L'anatomopathologie, anatomo-pathologie ou anatomie pathologique, informellement abrégée en « anat-patho » ou « anapath » dans le jargon des professionnels de la santé, est une spécialité médicale humaine et vétérinaire.  = Liens externes =Définition d'affectionAfetiv, Blog sur les relations affectives (fr) Portail de la psychologie  Ce sont des associations d'affections cancéreuses et de maladies diverses.] Expliquez-moi le terme médical en mots simples, par une paraphrase ou une courte définition :affection néoplasique [/INST]",
 'input_ids': [2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,


In [33]:
model.eval()
device = 'cuda'
print("Generation started......")
for T in [50]:
    queries2 = []
    responses2 = []
    times2 = []

    for batch in tqdm(data_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        #print(input_ids.shape)
        t1 = time.time()
        outputs = model.generate(input_ids=input_ids, 
                             attention_mask = attention_mask,
                             max_new_tokens=T)
        #print(outputs)
        answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        t2 = time.time()
        responses2.extend(answer)
        times2.extend([(t2-t1)/input_ids.shape[0]]*input_ids.shape[0])
        #break

    pd.DataFrame({"prompt":tokenized_testdata['prompt'], 
                      "raga": responses2, 
                      "time_taken":times2,
                 "src_docs": src_docs}).to_csv(f'./final_gptq_{enc}_biom_p4-{split}-t{T}.csv', sep='\t', index=False) 




Generation started......


100%|██████████| 40/40 [48:32<00:00, 72.81s/it]


In [32]:
pd.DataFrame({"prompt":tokenized_testdata['prompt'], 
                      "raga": responses2, 
                      "time_taken":times2,
                 "src_docs": src_docs}).to_csv(f'./final_gptq_{enc}_biom_p4-{split}-t{T}.csv', sep='\t', index=False) 